In [1]:
import numpy as np
import pandas as pd 
import requests
import math

# Install faiss
import faiss

In [2]:
df = pd.read_csv("books.csv")
df.head()
#PRINT THE COUNT OF EACH CATGORY
# df['categories'].value_counts()
x = df.copy()
x = x.loc[:999]
x['average_rating'] = np.round(x['average_rating'])
bins = list(range(0, int(x['num_pages'].max()),100))
x['num_pages'] = np.digitize(x['num_pages'], bins)
x['num_pages'] = x['num_pages'].apply(lambda x: bins[x-1])
x.to_csv('books_1000.csv', index=False)




In [4]:
books = df.copy()
books = books.drop(columns = ['isbn13','isbn10','thumbnail'], axis = 1)
b = books.isna().sum()
b
# books.columns

title                0
subtitle          4429
authors             72
categories          99
description        262
published_year       6
average_rating      43
num_pages           43
ratings_count       43
dtype: int64

In [5]:
def textual_representation(row):
    return f"""Title: {row['title']}
Subtitle: {row['subtitle']}
Authors: {row['authors']}
Categories: {row['categories']}
Description: {row['description']}
Published Year: {row['published_year']}
Average Rating: {row['average_rating']}
Num Pages: {row['num_pages']}
Rating Count: {row['ratings_count']}
"""


In [8]:
books['textual_representation'] = books.apply(textual_representation, axis = 1)
x = books.copy()
x = x.loc[:999]
x['average_rating'] = np.round(x['average_rating'])
bins = list(range(0, int(x['num_pages'].max()),100))
x['num_pages'] = np.digitize(x['num_pages'], bins)
x['num_pages'] = x['num_pages'].apply(lambda x: bins[x-1])
x['authors'] = x['authors'].replace(np.nan, 'Unknown')
x['categories'] = x['categories'].replace(np.nan, 'Unknown')
x['description'] = x['description'].replace(np.nan, 'Unknown')
x['published_year'] = x['published_year'].replace(np.nan, 'Unknown')
x['ratings_count'] = x['ratings_count'].replace(np.nan, '0')
x['average_rating'] = x['average_rating'].replace(np.nan, '0')
x['average_rating'] = np.round(x['average_rating'])
bins = list(range(0, int(x['num_pages'].max()),100))
x['num_pages'] = np.digitize(x['num_pages'], bins)
x['num_pages'] = x['num_pages'].apply(lambda x: bins[x-1])

x.columns
x.to_csv('b.csv', index=False)



In [7]:
dim = 3072
index = faiss.IndexFlatL2(dim)
# embedings = np.zeros((len(books['textual_representation']), dim), dtype = np.float32)
embeddings = np.zeros((1000,dim), dtype = np.float32)
print(embeddings.shape)


(1000, 3072)


In [48]:
for i, representation in enumerate(books['textual_representation']):
    if i%100 == 0:  
        print(f"Processing {i}th book")
    if i == 1000:
        break
    url = 'http://localhost:11434/api/embeddings'
    body = {
        "model" : "llama3.2",
        "prompt" : representation
    }
    request = requests.post(url, json = body)
    embeding = request.json()['embedding']
    print(embeding)
    print(len(embeding))
    print(np.array(embeding))
    print(np.array(embeding).shape)

    break

    embeddings[i] = np.array(embeding)
index.add(embeddings)


Processing 0th book
[1.204535722732544, -0.7159171104431152, -1.9553003311157227, 2.442530393600464, -0.049289919435977936, 1.4652953147888184, -0.08234553039073944, 1.240243911743164, -0.734896183013916, 0.22666293382644653, -3.199308156967163, 2.3167645931243896, -1.3793085813522339, 1.025016188621521, -0.6516571640968323, 0.19796520471572876, 1.0143340826034546, -1.1832131147384644, -2.6010360717773438, -1.0626710653305054, -0.5543898344039917, -0.09960146248340607, 0.8898250460624695, 3.1526949405670166, 0.9930910468101501, -0.639179527759552, 0.15164588391780853, -0.3158896863460541, 1.647842526435852, 0.22997665405273438, 0.7912859320640564, 1.2150883674621582, -0.3217249810695648, -0.3570910394191742, 0.33601638674736023, -0.46938538551330566, 1.3371139764785767, 0.5249863862991333, -0.22718364000320435, -1.7908445596694946, -0.31064462661743164, 0.554648220539093, 0.3698348104953766, -0.3841952085494995, -0.1272079050540924, 0.9281172156333923, -0.9611498713493347, 1.3752554655

In [9]:
faiss.write_index(index, "books.index")

In [27]:
index = faiss.read_index("books.index")
# emb = index.reconstruct(0)
# emb.shape

In [28]:
favorite_book = books.iloc[0]
print(favorite_book)

title                                                                Gilead
subtitle                                                                NaN
authors                                                  Marilynne Robinson
categories                                                          Fiction
description               A NOVEL THAT READERS and critics have been eag...
published_year                                                       2004.0
average_rating                                                         3.85
num_pages                                                             247.0
ratings_count                                                         361.0
textual_representation    Title: Gilead\nSubtitle: nan\nAuthors: Marilyn...
Name: 0, dtype: object


In [29]:
url = 'http://localhost:11434/api/embeddings'
body = {
    "model" : "llama3.2",
    "prompt" : favorite_book['textual_representation']
}
request = requests.post(url, json = body)

In [30]:
embedding = request.json()['embedding']
embedding = np.array(embedding).reshape(1, -1)
print(embedding.shape)

(1, 3072)


In [42]:
distance, indices = index.search(embedding, 5)
indices.flatten().tolist()

[0, 390, 87, 343, 929]

In [59]:
# best_matches  = np.array(books['textual_representation'])[index.flatten()]
best_matches = books['textual_representation'].iloc[[0,3]]

# for match in best_matches:
#     print(match)
#     print()
#     print("***********")

Title: Gilead
Subtitle: nan
Authors: Marilynne Robinson
Categories: Fiction
Description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s detail